# Add needed parameters to site files to run GMPE for Europe

### Notes
This notebook should be run after running 3_oq_vs30_uniform_grid.ipynb where the site and station_site files are creates

It brings most of the scripts and code from https://gitlab.seismo.ethz.ch/efehr/esrm20_sitemodel

##  Configuration

In [1]:
# Import dependencies

import glob
import numpy as np
import pandas as pd


import geopandas as gpd
from shapely.geometry import Point
import contextily as ctx
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

import utils
from plots import gdf_epicentre, stations_plot
from openquake.commands.prepare_site_model import main as oq_site_model

import os
import warnings
import h5py
import argparse
from scipy import stats
from scipy.interpolate import RegularGridInterpolator
from pyproj import CRS, Transformer
from shapely.geometry import Point

/Users/chago/openquake/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


## User input

In [2]:
# Events list from ECD
event = 'DRAFT_20110511_M5.1_Lorca'

# Add any reference rupture
rupture = 'earthquake_rupture_model_USGS.xml'

## Create region bounding box

### Get file

In [3]:
# Select files
# Select rupture file and get folders
file_path = glob.glob(os.path.join('..', '*', event, '*', rupture))[0]
folder = file_path[:file_path.find('Rupture')]

out_folder = os.path.join(folder, 'OpenQuake_gmfs')
print(out_folder)

# Read files
file_stations=os.path.join(out_folder,'site_model_stations.csv')
file_site_model=os.path.join(out_folder,'site_model.csv')

aux_file=os.path.join(out_folder,'aux_file.csv')

output_file_stations=os.path.join(out_folder,'site_model_stations_europe.csv')
output_file_site_model=os.path.join(out_folder,'site_model_europe.csv')

../Spain/DRAFT_20110511_M5.1_Lorca/OpenQuake_gmfs


In [4]:
# Data files
DATA_PATH = os.path.join('..','esrm20_sitemodel','exposure2site','site_data')
DATA_PATH

'../esrm20_sitemodel/exposure2site/site_data'

In [5]:
# Data files
DATA_PATH = os.path.join('..','esrm20_sitemodel','exposure2site','site_data')

# Geology shapefiles
GEOLOGY_FILE = os.path.join(DATA_PATH, "GEOL_V8_ERA2.shp")
# File containing 30 arc second grid of slope, geology, elevation and
# built density
# BBOX = [-25.0, 34.0, 45.0, 72.0]
SITE_HDF5_FILE = os.path.join(DATA_PATH, "europe_gebco_topography.hdf5")
# File containing Vs30s at 30 arcseconds using the USGS Wald & Allen method
# BBOX = [-30;, 30., 65., 75.]
VS30_HDF5_FILE = os.path.join(DATA_PATH, "europe_2020_vs30.hdf5")
# Pre-computed grid of volcanic front distances at 5 arc-minutes
# BBOX = [-25.0, 34.0, 45.0, 72.0]
VF_DIST_FILE = os.path.join(DATA_PATH, "volcanic_front_distances_5min.hdf5")
# Europe slice of 250 m x 250 m GHS built density dataset (in Mollewiede proj.)
BUILT_ENV_FILE = os.path.join(DATA_PATH, "ghs_global_250m_Europe.hdf5")
# ESHM20 Attenuation Region File
REGION_FILE = os.path.join(DATA_PATH, "ATTENUATION_REGIONS.shp")

In [6]:
class SiteManager(object):
    """
    General object for controlling the flow of the site model rendering.

    For handling the output site model it is preferble to use a pandas
    dataframe, unless the density is so large that memory usage makes this
    impossible

    Site model dataframe requires the following elements:

    "lons": longitudes (decimal degrees)
    "lats": latitudes (decimal degrees)
    "slope": slope (m/s)
    "geology": geological era
    "vs30": Vs30 inferred from topography unless set otherwise
    "xvf": Distance from the volcanic front (km)
    """
    REQUIRED_HEADERS = ["lon", "lat", "slope", "geology", "vs30",
                        "vs30measured", "xvf"]
    STRING_ATTRIBS = ["geology", ]

    def __init__(self, site_model, **kwargs):
        """
        """
        assert isinstance(site_model, gpd.GeoDataFrame)
        # Store sites as a geopandas Geodataframe
        self.site_model = site_model
        site_model_crs = kwargs.get("site_model_crs", "epsg:4326")
        if not self.site_model.crs:
            self.site_model.crs = {"init": site_model_crs}
        self.bbox = self.site_model.total_bounds
        self.default_conditions = {
            "vs30": kwargs.get("default_vs30", 800.0),
            "vs30measured": kwargs.get("vs30measured", 0),
            "z1pt0": kwargs.get("default_z1pt0", 31.7),
            "z2pt5": kwargs.get("default_z2pt5", 0.57),
            "xvf": kwargs.get("xvf", 150.),
            "region": kwargs.get("region", 0)}
        # Get the ESHM regions file and re-project to xy
        self.eshm20_regions = gpd.GeoDataFrame.from_file(REGION_FILE)
        self.eshm20_regions.crs = {"init": "epsg:4326"}
        self.eshm20_regions_xy = self.eshm20_regions.to_crs(
            {"init": EUROPE_EQUAL_AREA})
        # Assign ESHM20 regions
        self.get_eshm20_region()

    def __len__(self):
        return self.site_model.shape[0]

    def __repr__(self):
        return str(self.site_model)

    def __iadd__(self, model):
        # Adds a second site model concatenating the two dataframes
        self.site_model = pd.concat([self.site_model, model.site_model])
        # Drop duplicates
        self.site_model.drop_duplicates(["lon", "lat"], inplace=True)
        return self

    def __getitem__(self, key):
        return self.site_model[key].values

    def __add__(self, model):
        output_model = pd.concat([self.site_model, model.site_model])
        output_model.drop_duplicates(["lon", "lat"], inplace=True)
        return self(output_model)

    def plot(self, key, **kwargs):
        # Plot the site model using thr GeoDataFrame.plot() method
        self.site_model.plot(key, **kwargs)

    def get_eshm20_region(self):
        """
        Determines the ESHM20 attenuation region for the respective sites and
        adds to the site model
        """
        print("Assigning the sites to the ESHM20 regions")
        # Retrieve lightweight dataframe with just the geometry and re-project
        # the cartesian reference frame
        sm_lite = self.site_model[["lat", "lon", "geometry"]].to_crs(
            {"init": EUROPE_EQUAL_AREA})
        # Get the region assignments
        assignments = gpd.sjoin(sm_lite, self.eshm20_regions_xy, how="left")
        if assignments.shape[0] > self.site_model.shape[0]:
            # This can happen if the region polygons are overlapping (which
            # they shouldn't be in this case)
            assignments.drop_duplicates(["lon", "lat"], inplace=True)
        # Set any nans to region 0
        eshm20_region = assignments["REGION"].values
        eshm20_region[np.isnan(eshm20_region)] = 0.0
        self.site_model["region"] = eshm20_region.astype(int)

    @classmethod
    def site_model_from_bbox(cls, bbox, spacing, weighting, **kwargs):
        """
        Defines a site model from a bounding box, downsampled at the given
        spacing
        """
        if (spacing % 30):
            raise ValueError("Spacing must be a multiple of 30")
        num = int(spacing) // 30
        assert bbox[2] > bbox[0]
        assert bbox[3] > bbox[1]
        averaging = kwargs.get("averaging", "MEAN")
        geol_weighting = kwargs.get("geol_weighting", "MODE")
        default_xvf = kwargs.get("default_xvf", 150.0)
        smooth_n = kwargs.get("smooth_n", 0)
        onshore_only = kwargs.get("onshore_only", True)
        as_dataframe = kwargs.get("as_dataframe", True)
        return cls(
            downsample_slope_geology_vs30_grid(bbox, num, weighting,
                                               averaging=averaging,
                                               geol_weighting=geol_weighting,
                                               default_xvf=default_xvf,
                                               n=smooth_n,
                                               onshore_only=onshore_only,
                                               as_dataframe=as_dataframe)
        )

    @classmethod
    def site_model_from_exposure_model(cls, admin_model, admin_level,
                                       weighting=True, averaging="MEAN",
                                       **kwargs):
        """
        Defines the site model from a set of polygons in the form of a
        geopandas.GeoDataFrame, taking the "average" site property across
        the entire polygon
        """
        geol_weighting = kwargs.get("geol_weighting", "MODE")
        default_xvf = kwargs.get("default_xvf", 150.0)
        smooth_n = kwargs.get("smooth_n", 0)
        # Build exposure geodataframe
        print("---- Calculating site properties")
        admin_model = get_slope_geology_vs30_polygons(
            admin_model, "ID_{:g}".format(admin_level), weighting,
            method=averaging, method_geol=geol_weighting, n=smooth_n,
            proj=EUROPE_EQUAL_AREA)
        # Get the backarc distances
        print("---- Getting backarc distance")
        admin_model["xvf"] = interpolate_xvf_grid(
            admin_model["lon"].values, admin_model["lat"].values,
            default_xvf=default_xvf)
        # Should now have a geometry (polygon), lons, lats, vs30s, geologies,
        # slopes and backarc distances - so basically a fully formed site
        # model
        return cls(admin_model, **kwargs)

    @classmethod
    def site_model_from_points(cls, lons, lats, **kwargs):
        """
        Defines the site model by assigning to each location the site property
        at the exact location of the site
        """
        onshore_only = kwargs.get("onshore_only", True)
        site_model = get_slope_geology_vs30_at_location(
            lons, lats, spc=(1. / 120.),
            onshore_only=onshore_only,
            as_dataframe=True)
        return cls(site_model, **kwargs)

    @classmethod
    def site_model_from_admin_zones(cls, admin_model, admin_level,
                                    weighting=True, centroid="MEAN",
                                    averaging="MEAN", **kwargs):
        """
        Similar to the case of the exposure model; however the starting point
        is a set of polygons without associated sites, and thus the centroids
        need to be defined.
        """
        geol_weighting = kwargs.get("geol_weighting", "MODE")
        default_xvf = kwargs.get("default_xvf", 150.0)
        smooth_n = kwargs.get("smooth_n", 0)
        print("---- Determining centroids")
        admin_model = get_weighted_centroids_of_polygon_wgs84(
            admin_model, admin_level, centroid_method=centroid, n=smooth_n)
        print("---- Getting site properties")
        admin_model = get_slope_geology_vs30_polygons(
            admin_model, "ID_{:g}".format(admin_level), weighting,
            method=averaging, method_geol=geol_weighting, n=smooth_n,
            proj=EUROPE_EQUAL_AREA)
        print("---- Getting backarc distance")
        admin_model["xvf"] = interpolate_xvf_grid(
            admin_model["lon"].values, admin_model["lat"].values,
            default_xvf=default_xvf)
        return cls(admin_model, **kwargs)

    @staticmethod
    def build_amplification_model(bbox, spacing, imts, reference_slope,
                                  reference_geology, skip_unknown=True,
                                  **kwargs):
        """
        Builds model of amplification for a given region defined by a bounding
        box

        :param list bbox:
            Bounding box to define the region [east, south, west, north]
        :param float spacing:
            Spacing of the model in arc-seconds (must be a multiple of 30)
        :param list imts:
            Intensity measure types as a list containing either "pga" or
            float values representing the periods, e.g. ["pga", 0.1, 0.2, 1.0]
        :param float reference_slope:
            Slope value (in m/m) to be adopted as the reference slope
        :param str reference_geology:
            Geological unit to be adopted as the reference geology
        :param bool skip_unknown:
            For sites with an unknown geological unit (including offshore)
            return a NaN
        **kwargs = Other arguments relevant for downsampling passed to
                   downsample_slope_geology_vs30_grid

        :returns:
            Amplification model as a dictionary containing:
            'amplification': 3D grid of amplification [nlat, nlon, num. imts]
            'phis2s': Site-to-site standard deviation (phis2s) as a list of
                      length len(imts)
            'lons': Vector of longitudes
            'lats': Vector of latitudes
            'slope': 2D array of slope values (in m/m)
            'geology': 2D numpy array of geological units
            'bbox': Bounding box information
            'spacing': spacing information
            'imts': List of intensity measured
        """
        assert bbox[2] > bbox[0]
        assert bbox[3] > bbox[1]
        if (spacing % 30):
            raise ValueError("Spacing must be a multiple of 30")
        num = int(spacing) // 30
        if num == 1:
            # Extracting the 30 arc-second model so no downsampling is needed
            lons, lats, slope = slice_wgs84_datafile(SITE_HDF5_FILE, bbox, "slope")
            geology_code = slice_wgs84_datafile(SITE_HDF5_FILE, bbox, "geology")[2]

        else:
            # Downsampling the 30 arc second grid to a lower resolution
            weighting = kwargs.get("weighting", True)
            averaging = kwargs.get("averaging", "MEAN")
            geol_weighting = kwargs.get("geol_weighting", "MODE")
            default_xvf = kwargs.get("default_xvf", 150.0)
            smooth_n = kwargs.get("smooth_n", 0)
            slope, vs30s, geology_code, xvf, lons, lats, idx =\
                downsample_slope_geology_vs30_grid(bbox, num, weighting,
                                                   averaging=averaging,
                                                   geol_weighting=geol_weighting,
                                                   default_xvf=default_xvf,
                                                   n=smooth_n,
                                                   onshore_only=False,
                                                   as_dataframe=False)
        # Convert geology from the code to the expected bytestrings
        geology = np.zeros(geology_code.shape, dtype=(np.string_, 20))
        for key in GEOL_DICT_KEY_TO_NAME:
            idx = geology_code == key
            if np.any(idx):
                geology[idx] = GEOL_DICT_KEY_TO_NAME[key]
        ampls, phis2s = site_amp_model_slope_geology(
            imts, slope, geology,
            reference_slope=reference_slope,
            reference_geology=reference_geology,
            skip_unknown=True)
        for i, (imt, phis2s_i) in enumerate(zip(imts, phis2s)):
            print("IMT: %s    PHI_S2S = %.6f" % (str(imt), phis2s_i))
        return {"amplification": ampls, "phis2s": phis2s, "lons": lons,
                "lats": lats, "slope": slope, "geology": geology,
                "bbox": bbox, "spacing": spacing, "imts": imts}

    @staticmethod
    def export_amplification_model(output_folder, model, filetype="tif"):
        """
        Exports an amplifiction model to a set of 2D raster files

        :param str output_folder:
            Path to output folder for export (will be created if it doesn't exist
        :param dict model:
            Amplification model as output from SiteManager.build_amplification_model
        :param str filetype:
            File type to export the rasters: geotiff ('tif') or ESRI arc-ascii ('asc')
        """
        spc = float(int(model["spacing"]) // 30) * (1.0 / 120.0)
        if not os.path.exists(output_folder):
            os.mkdir(output_folder)
        if filetype == "tif":
            export_amplification_maps_to_geotiff(
                output_folder,
                model["amplification"],
                model["imts"],
                model["bbox"],
                spc, spc
                )
        elif filetype == "asc":
            export_amplification_maps_to_ascii(
                output_folder,
                model["amplification"],
                model["imts"],
                model["bbox"],
                spc, spc
                )
        else:
            raise OSError("File type for export %s not recognised" % filetype)
        return

    def replace_geometry_for_lonlat(self, output_model):
        """
        Replaces the Point geometry with the longitude and latitude
        """
        lonlat = np.array([[geom.x, geom.y]
                           for geom in self.site_model.geometry.values])
        output_model["lon"] = lonlat[:, 0]
        output_model["lat"] = lonlat[:, 1]
        # Remove the original geometry (makes the output lighter)
        del output_model["geometry"]
        return output_model

    def add_backarc_distances_to_site_model(self, default_xvf=150.):
        """
        Retrieves the volcanic front distance for the target sites
        """
        if "xvf" in self.site_model.columns:
            return
        self.site_model["xvf"] =\
            interpolate_xvf_grid(
                self.site_model["lon"].values, self.site_model["lat"].values,
                default_xvf=default_xvf)

    def replace_lonlat_for_geometry(self, output_model):
        """
        Replaces the longitude and latitude with the geometry
        """
        output_model["geometry"] = gpd.GeoSeries(
           [Point(lon, lat) for lon, lat in zip(self.site_model["lon"].values,
                                                self.site_model["lat"].values)])
        # Delete the lon, lat
        del output_model["lon"]
        del output_model["lat"]
        return output_model

    def insert_default_column(self, output_model, default_key):
        """
        Adds a column of default values to the site model
        """
        default_val = self.default_conditions[default_key]
        if isinstance(default_val, str):
            # Add the list of strings
            output_model[default_key] =\
                [default_val for i in range(self.site_model.shape[0])]
        else:
            # Is numerical - add a numpy array
            output_model[default_key] =\
                default_val * np.ones(self.site_model.shape[0],
                                      dtype=default_val.__class__)

    def to_csv(self, filename, sep=","):
        """
        Exports the site model to a csv format
        """
        output_model = self.site_model.copy()
        # Check headers are correct
        invalid_points = np.zeros(output_model.shape[0], dtype=int)
        for col in self.REQUIRED_HEADERS:
            if col not in self.site_model.columns:
                if col in ("lon", "lat"):
                    # Lon/Lat not in site model - can retrieve it from geometry
                    output_model = self.replace_geometry_for_lonlat(
                        output_model)
                elif col in ("vs30measured",):
                    self.insert_default_column(output_model, "vs30measured")
                else:
                    raise ValueError("Site model missing attribute %s" % col)
            # Check for invalid values, i.e. nans
            invalid_points += output_model[col].isnull().values.astype(int)
        idx = invalid_points > 0
        if np.any(invalid_points):
            # Purge sites with nans in the required attributes
            output_model = output_model[np.logical_not(idx)]
        # Drop duplicates
        output_model.drop_duplicates(["lon", "lat"], inplace=True)
        print("Exporting model to csv file %s" % filename)
        if "geometry" in output_model.columns:
            # Remove geometry
            del output_model["geometry"]
        output_model.to_csv(filename, sep=sep, index=False)

    @classmethod
    def from_csv(cls, filename, sep=",", **kwargs):
        """
        Builds a site model object from a csv file
        """
        site_model_input = pd.read_csv(filename, sep=",")
        site_model = gpd.GeoDataFrame({
            "geometry": gpd.GeoSeries(
                [Point(lon, lat) for lon, lat in zip(site_model_input["lon"].values,
                                                     site_model_input["lat"].values)]
                ),
            }
        )

        for key in cls.REQUIRED_HEADERS:
            if key not in site_model_input.columns:
                print("Required attribute %s missing from file - may not "
                      "function as needed" % key)
            else:
                site_model[key] = site_model_input[key]
        return cls(site_model, **kwargs)

    def to_shapefile(self, filename):
        """
        Exports the site model to shapefile
        """
        output_model = self.site_model.copy()
        for col in self.REQUIRED_HEADERS:
            if col not in self.site_model.columns:
                if col in ("vs30measured",):
                    self.insert_default_column(output_model, "vs30measured")
                else:
                    raise ValueError("Site model missing attribute %s" % col)

        if "geometry" not in self.site_model.columns:
            output_model = self.replace_lonlat_for_geometry(output_model)

        print("Exporting site model to shapefile %s" % filename)
        output_model.to_file(filename)

    @classmethod
    def from_shapefile(cls, filename, **kwargs):
        """
        Loads in the site model from a shapefile
        """
        input_model = gpd.GeoDataFrame.from_file(filename)
        has_lonlat = ("lon" in input_model.columns) and\
            ("lat" in input_model.columns)
        if not has_lonlat:
            # Longitude and latitude columns missing; take them from geometry
            lons, lats = zip(*[(pnt.x, pnt.y)
                             for pnt in input_model.geometry.values])
            input_model["lon"] = np.array(lons)
            input_model["lat"] = np.array(lats)

        for key in cls.REQUIRED_HEADERS:
            if key not in input_model.columns:
                print("Required attribute %s missing from file - may not "
                      "function as needed" % key)
        return cls(input_model, **kwargs)

    def to_xml(self, filename, fmt="%s"):
        """
        Exports the site model to a site model xml file
        """
        output_model = self.site_model.copy()
        invalid_points = np.zeros(output_model.shape[0], dtype=int)
        # Check headers are correct
        for col in self.REQUIRED_HEADERS:
            if col not in self.site_model.columns:
                if col in ("lon", "lat"):
                    # Lon/Lat not in site model - can retrieve it from geometry
                    output_model = self.replace_geometry_for_lonlat(
                        output_model)
                elif col in ("vs30measured",):
                    self.insert_default_column(output_model, "vs30measured")
                else:
                    raise ValueError("Site model missing attribute %s" % col)
            # Check for invalid values, i.e. nans
            invalid_points += output_model[col].isnull().values.astype(int)
        idx = invalid_points > 0
        if np.any(invalid_points):
            # Purge sites with nans in the required attributes
            output_model = output_model[np.logical_not(idx)]
        # Drop duplicate sites
        output_model.drop_duplicates(["lon", "lat"], inplace=True)
        nodes = []
        print("Building site_model for export")
        for i in range(output_model.shape[0]):
            attribs = {"lon": output_model["lon"].values[i],
                       "lat": output_model["lat"].values[i],
                       "vs30": output_model["vs30"].values[i],
                       "slope": output_model["slope"].values[i],
                       "geology": output_model["geology"].values[i],
                       "xvf": output_model["xvf"].values[i],
                       "region": output_model["region"].values[i]}
            if output_model["vs30measured"].values[i]:
                attribs["vs30Type"] = "measured"
            else:
                attribs["vs30Type"] = "inferred"

            nodes.append(Node("site", attrib=attribs))
        site_model = Node("siteModel", nodes=nodes)
        print("Exporting site model to xml file %s" % filename)
        with open(filename, "wb") as f:
            nrml_write([site_model], f, fmt=fmt)

    @classmethod
    def from_xml(cls, filename, **kwargs):
        """
        Builds a site model object from a nrml file
        """
        [site_input] = nrml_read(filename)
        site_model = {}
        for node in site_input:
            if not site_model:
                for key in node.attrib:
                    if key in cls.STRING_ATTRIBS:
                        site_model[key] = [node[key]]
                    elif key == "vs30Type":
                        if node[key] == "measured":
                            site_model["vs30measured"] = [1]
                        else:
                            site_model["vs30measured"] = [0]

                    else:
                        site_model[key] = [float(node[key])]
            else:
                for key in node.attrib:
                    if key in cls.STRING_ATTRIBS:
                        site_model[key].append(node[key])
                    elif key == "vs30Type":
                        if node[key] == "measured":
                            site_model["vs30measured"].append(1)
                        else:
                            site_model["vs30measured"].append(0)
                    else:
                        site_model[key].append(float(node[key]))
        for key in site_model:
            if key not in cls.STRING_ATTRIBS:
                site_model[key] = np.array(site_model[key])
        site_model["geometry"] = gpd.GeoSeries(
            [Point(lon, lat) for lon, lat in zip(site_model["lon"],
                                                 site_model["lat"])]
        )
        return cls(gpd.GeoDataFrame(site_model), **kwargs)


def set_up_arg_parser():
    """
    """
    description = ("Builds the site models needed for SERA seismic risk "
                   "calculations, from the 30 arc second grids. To run type: \n"
                   "\n"
                   ">> python exposure_to_site_tools.py --run CALC_TYPE "
                   "--output-file PATH/TO/OUTPUT_FILE\n"
                   "\n"
                   "Other arguments are specific to the type of calculator\n")
    parser = argparse.ArgumentParser(
        description=description, add_help=True,
        formatter_class=argparse.RawTextHelpFormatter)
    parser.add_argument('-r', '--run',
                        choices=["grid", "admin", "exposure", "point", "join",
                                 "amplification"],
                        help="Type of configuration ('grid', "
                        "'admin', 'exposure', 'point', 'join', 'amplification')",
                        required=True)
    parser.add_argument('-of', '--output-file', help='Path to output file',
                        required=True)

    # Options applicable to many groups
    parser.add_argument('-w', '--weighting', help="Use building density to weight "
                        "the values within an area/cell (default True)",
                        required=False, default="True")
    parser.add_argument('-a', '--averaging', help="Defines how to take assign a "
                        "value within a cell or polygon ('mean', 'median', max')",
                        choices=["mean", "median", "max"],
                        required=False, default="mean")
    parser.add_argument('-gw', '--geological-weighting', help="Get the weighted "
                        "median geological category (or else the modal "
                        "category)", choices=["mode", "median", "max"],
                        required=False, default="mode")
    parser.add_argument('-dxvf', '--default-xvf',
                        help="Default distance to the volcanic front  (km)",
                        required=False, default="150.0")
    parser.add_argument('-oo', '--onshore-only', help="Clip the grid sites to "
                        "only those onshore", required=False,
                        default="True")
    parser.add_argument('-sn', '--smooth-n', help="N-th order neighbourhood to"
                        " use for weighting", required=False, default="0")

    # Arguments specific to the regular grid option
    grid_group = parser.add_argument_group(
        "grid",
        "Renders site model on a regularly-spaced, weighted grid")
    grid_group.add_argument('--bbox', help="Bounding Box LLON/LLAT/ULON/ULAT",
                            required=False, default=None)
    grid_group.add_argument('-s', '--spacing', help="Spacing of downsampled grid "
                            "in arc seconds (must be a multiple of 30)",
                            required=False, default="30")

    # Arguments specific to the exposure model option
    exposure_group = parser.add_argument_group(
        "exposure",
        "Builds site model by averaging across polygons in an exposure model")
    exposure_group.add_argument('-iexp', '--input-exposure',
                                help="Path to input csv exposure file",
                                required=False, default=None)
    exposure_group.add_argument('-sd', '--shapefile-dir',
                                help="Path to directory containing shapefiles",
                                required=False, default=None)
    exposure_group.add_argument('-tal', "--target-admin-level",
                                help="Target admin level for the exposure "
                                "- highest in exposure file if not specified",
                                required=False, default=None)

    # Arguments specific to the admin regions option
    admin_group = parser.add_argument_group(
        "admin",
        "Builds a site model for a set of admin regions, including definition "
        "of the centroid")
    admin_group.add_argument('-ishp', "--input-shapefile",
                             help="Path to input admin shapefile",
                             required=False)
    admin_group.add_argument('-al', "--admin-level",
                             help="Administrative level for aggregation (will "
                             "take the highest found if not specified)",
                             required=False, default="")
    admin_group.add_argument('-cw', "--centroid-weighting",
                             help="Method for weighting the centroid of the "
                             "admin region according to building density",
                             choices=["mean", "median", "max"],
                             required=False, default="")

    # Arguments specific to the point model option
    point_group = parser.add_argument_group(
        "point",
        "Builds site model from a set of locations by taking the property "
        "value directly at the location"
        )
    point_group.add_argument('-if', "--input-file",
                             help="Path to input text/csv file",
                             required=False)

    # Arguments specific to the join model option
    join_group = parser.add_argument_group(
        "join",
        "Join together multiple site model files or a directory of files"
        )
    join_group.add_argument('-id', '--input-dir',
                            help="Path to directory of input files")
    join_group.add_argument('-f', '--files', help="+ delimited list of files "
                            "(e.g. file1.csv+file2.csv+...)")

    # Arguments specific to the amplification map option
    ampl_group = parser.add_argument_group(
        "amplification",
        "Builds a map of expected amplification for a given region "
        "(requires `bbox` and `spacing` arguments as for the `grid` workflow"
        )
    ampl_group.add_argument('-imts', '--imts', help="Comma separated list of intensity measure "
                            "types as either 'pga' or spectral period, e.g. for PGA, SA(0.1s),"
                            " SA(1.0 s) input 'pga,0.1,1.0", required=True)
    ampl_group.add_argument('-rs', '--reference-slope', help="Reference slope condition (m/m)",
                            required=True)
    ampl_group.add_argument('-rg', '--reference-geology', help="Reference geological unit "
                            "(must be one from the following: %s)" % str(GEOLOGICAL_UNITS),
                            required=True)
    ampl_group.add_argument('-ft', '--file-type', help="File type to export amplification "
                            "map: either Geotiff ('tif') or ESRI Arc-Ascii ('asc')",
                            required=False, default="tif")

    # Return arguments
    return parser


def main():
    parser = set_up_arg_parser()
    # Argparser - arguments
    args = parser.parse_args()
    # Check if output file already exists and raise error if so
    if os.path.exists(args.output_file):
        raise IOError("Output file %s already exists!" % args.output_file)

    if args.run == "amplification":
        os.mkdir(args.output_file)
    else:
        # Define output file type and check they are one of "csv", "xml" or "sha"
        output_file_type = os.path.splitext(args.output_file)[1]
        assert output_file_type in (".csv", ".xml", ".shp")

    if args.run == "grid":
        # ----------------- Workflow 1 ----------------------------------------
        # From a bounding box get a downsampled grid at a multiple of 30 arc
        # seconds
        bbox = list(map(float, args.bbox.split("/")))
        spacing = int(args.spacing)
        weighting = bool(args.weighting)
        averaging = args.averaging.upper()
        onshore_only = bool(args.onshore_only)
        geol_weighting = args.geological_weighting.upper()
        def_xvf = float(args.default_xvf)
        smooth_n = int(args.smooth_n)
        # Build the site model
        site_manager = SiteManager.site_model_from_bbox(
            bbox, spacing, weighting, averaging=averaging, geol_weighting=geol_weighting,
            default_xvf=def_xvf, onshore_only=onshore_only, smooth_n=smooth_n)

    elif args.run == "exposure":
        # ---------------- Workflow 2 -----------------------------------------
        # From an input exposure model file and a path to the corresponding
        # geometries in the shapefile, constructs a site model by averaging the
        # properties across the administrative polygon
        weighting = args.weighting
        averaging = args.averaging.upper()
        onshore_only = bool(args.onshore_only)
        geol_weighting = args.geological_weighting.upper()
        def_xvf = float(args.default_xvf)
        smooth_n = int(args.smooth_n)

        # Get the admin model (as a geodataframe) and the admin level from
        # the exposure model
        print("Retrieving exposure model from %s" % args.input_exposure)
        admin_model, admin_level = get_site_set_from_exposure(
            args.input_exposure, args.shapefile_dir,
            args.target_admin_level)
        print("Building site model")
        site_manager = SiteManager.site_model_from_exposure_model(
            admin_model, admin_level, weighting, averaging=averaging,
            geol_weighting=geol_weighting, default_xvf=def_xvf,
            smooth_n=smooth_n)

    elif args.run == "admin":
        # --------------- Workflow 3 ------------------------------------------
        # From a set of administrative boundaries defines the site model,
        # including the determination of the target sites from the [weighted]
        # centroids of each admin zone

        # Setup general properties
        weighting = args.weighting
        averaging = args.averaging.upper()
        geol_weighting = args.geological_weighting.upper()
        def_xvf = float(args.default_xvf)
        centroid_weighting = args.centroid_weighting.upper()
        smooth_n = int(args.smooth_n)
        # Admin level
        if args.admin_level:
            admin_level = int(args.admin_level)
            # Check this is in the model
        else:
            admin_level = None

        # Load in data from file
        print("Loading admin model %s" % args.input_shapefile)
        admin_model = gpd.GeoDataFrame.from_file(args.input_shapefile)
        # Just in case the ID and NAME columns are in lower case, render them
        # all to upper case
        admin_model = admin_model.rename(KEY_MAPPER, axis='columns')
        if admin_level:
            # Admin level requested but not found in shapefile
            if not "ID_{:g}".format(admin_level) in admin_model.columns:
                raise IOError("Required admin level %g not defined in "
                              "shapefile %s" % (admin_level,
                                                args.input_shapefile))
        else:
            admin_level = get_maximum_admin_level(admin_model)
            print("Highest admin level found = %g" % admin_level)
        # Now setup the site model
        print("Building site model")
        site_manager = SiteManager.site_model_from_admin_zones(
            admin_model, admin_level, weighting, centroid_weighting, averaging,
            geol_weighting=geol_weighting, default_xvf=def_xvf,
            smooth_n=smooth_n)

    elif args.run == "point":
        # --------------- Workflow 4 ------------------------------------------
        # For a set of longitudes and latitudes simply retrieve the properties
        # at the locations of the longitudes and latitudes

        onshore_only = bool(args.onshore_only)
        print("Loading locations from %s" % args.input_file)
        df = pd.read_csv(args.input_file, sep=",")
        has_lonlat = ("lon" in df.columns) and ("lat" in df.columns)
        if not has_lonlat:
            # Check if inputs in xcoord, ycoord
            has_crds = ("xcoord" in df.columns) and ("ycoord" in df.columns)
            if has_crds:
                df["lon"] = df["xcoord"].values
                df["lat"] = df["ycoord"].values
            else:
                raise ValueError("Input file %s has no lon,lat or "
                                 "xcoord,ycoord attributes" % args.input_file)
        # Dropping duplicate sites
        df.drop_duplicates(["lon", "lat"], inplace=True)
        print("Building site model for points")
        site_manager = SiteManager.site_model_from_points(
            df["lon"].values, df["lat"].values, onshore_only=onshore_only)

    elif args.run == "join":
        if args.input_dir and os.path.isdir(args.input_dir):
            files = []
            for filename in os.listdir(args.input_dir):
                if filename.endswith(".csv") or filename.endswith(".xml"):
                    files.append(os.path.join(args.input_dir, filename))
            if not len(files):
                raise ValueError("No csv or xml files found in %s"
                                 % args.input_dir)
        else:
            files = args.files.split("+")
        assert len(files) > 1

        if files[0].endswith("csv"):
            site_manager = SiteManager.from_csv(files[0])
        elif files[0].endswith("xml"):
            site_manager = SiteManager.from_xml(files[0])
        else:
            raise ValueError("File %s not csv or xml" % files[0])
        for filename in files[1:]:
            if filename.endswith("csv"):
                temp_manager = SiteManager.from_csv(filename)
            elif filename.endswith("xml"):
                temp_manager = SiteManager.from_xml(filename)
            else:
                raise ValueError("File %s not csv or xml" % filename)
            site_manager += temp_manager

    elif args.run == "amplification":
        bbox = list(map(float, args.bbox.split("/")))
        spacing = int(args.spacing)
        weighting = bool(args.weighting)
        averaging = args.averaging.upper()
        onshore_only = bool(args.onshore_only)
        geol_weighting = args.geological_weighting.upper()
        def_xvf = float(args.default_xvf)
        smooth_n = int(args.smooth_n)

        imts = [imt if imt in ("pga", "pgv") else float(imt)
                for imt in args.imts.split(",")]

        amplification = SiteManager.build_amplification_model(
            bbox, spacing, imts, float(args.reference_slope),
            args.reference_geology, skip_unknown=onshore_only, weighting=weighting,
            averaging=averaging, geol_weighting=geol_weighting,
            default_xvf=def_xvf, smooth_n=smooth_n)
        SiteManager.export_amplification_model(args.output_file,
                                               amplification,
                                               args.file_type)
        return
    else:
        # Shouldn't really have reached this point, but raising an IOError
        # just in case
        raise IOError("Run type %s not recognised" % args.run)
    print("Exporting to file %s" % args.output_file)
    # Export
    if output_file_type == ".xml":
        # Export to xml
        site_manager.to_xml(args.output_file)
    elif output_file_type == ".shp":
        site_manager.to_shapefile(args.output_file)
    else:
        # Export to csv
        site_manager.to_csv(args.output_file)
    return

In [7]:
# Coordinate reference systems
CRS_WGS84 = CRS.from_epsg(4326)
MOLL_STR = '+ellps=WGS84 +lon_0=0 +no_defs +proj=moll +units=m +x_0=0 +y_0=0'
CRS_MOLL = CRS.from_string(MOLL_STR)
EUROPE_EQUAL_AREA = "epsg:3035"
SMOOTHED_N = [1, 3, 5]
# Transformations between Molleweide and WGS84
TRANSFORMER = Transformer.from_crs(CRS_WGS84, CRS_MOLL, always_xy=True)
TRANSFORMER_REV = Transformer.from_crs(CRS_MOLL, CRS_WGS84, always_xy=True)

# Geological eras stored as integers - relates the geological era to the int
GEOL_DICT_KEY_TO_NAME = {0: "UNKNOWN",
                         1: "PRECAMBRIAN",
                         2: "PALEOZOIC",
                         3: "JURASSIC-TRIASSIC",
                         4: "CRETACEOUS",
                         5: "CENOZOIC",
                         6: "PLEISTOCENE",
                         7: "HOLOCENE"}


GEOL_DICT_NAME_TO_KEY = {"UNKNOWN": 0,
                         "PRECAMBRIAN": 1,
                         "PALEOZOIC": 2,
                         "JURASSIC-TRIASSIC": 3,
                         "CRETACEOUS": 4,
                         "CENOZOIC": 5,
                         "PLEISTOCENE": 6,
                         "HOLOCENE": 7}

def get_eshm20_region(self):
        """
        Determines the ESHM20 attenuation region for the respective sites and
        adds to the site model
        """
        print("Assigning the sites to the ESHM20 regions")
        # Retrieve lightweight dataframe with just the geometry and re-project
        # the cartesian reference frame
        sm_lite = self.site_model[["lat", "lon", "geometry"]].to_crs(
            {"init": EUROPE_EQUAL_AREA})
        # Get the region assignments
        assignments = gpd.sjoin(sm_lite, self.eshm20_regions_xy, how="left")
        if assignments.shape[0] > self.site_model.shape[0]:
            # This can happen if the region polygons are overlapping (which
            # they shouldn't be in this case)
            assignments.drop_duplicates(["lon", "lat"], inplace=True)
        # Set any nans to region 0
        eshm20_region = assignments["REGION"].values
        eshm20_region[np.isnan(eshm20_region)] = 0.0
        self.site_model["region"] = eshm20_region.astype(int)
        
def site_model_from_points(lons, lats, **kwargs):
        """
        Defines the site model by assigning to each location the site property
        at the exact location of the site
        """
        onshore_only = kwargs.get("onshore_only", True)
        site_model = get_slope_geology_vs30_at_location(
            lons, lats, spc=(1. / 120.),
            onshore_only=onshore_only,
            as_dataframe=True)
        return cls(site_model, **kwargs)
    
def get_slope_geology_vs30_at_location(site_lons, site_lats, spc=(1. / 120.),onshore_only=True, as_dataframe=False):
    """
    Retrieves the slope and geology at a set of locations
    :param np.ndarray site_lons:
        Longitudes of target sites
    :param np.ndarray site_lats:
        Latitudes of target sites
    :param float spc:
        Spacing of reference hdf5 file with data (probably 120 arc seconds)
    :returns:
        site_slope: slopes of the site (in m/m)
        site_vs30: Inferred Vs30 (in m/s)
        site_geology: geological index of the site
        site_backarc_distance: backarc distance (in km)
    """
    # Get bounding box of sites (with spc buffer)
    bbox = [np.min(site_lons) - spc, np.min(site_lats) - spc,
            np.max(site_lons) + spc, np.max(site_lats) + spc]
    # Get slope and geology from hdf5 file
    print("---- Retrieving data within bounding box")
    lons, lats, slope = slice_wgs84_datafile(SITE_HDF5_FILE, bbox, "slope")
    geology = slice_wgs84_datafile(SITE_HDF5_FILE, bbox, "geology")[2]
    lons_vs, lats_vs, vs30 = slice_wgs84_datafile(VS30_HDF5_FILE, bbox, "vs30")
    if onshore_only:
        elevation = slice_wgs84_datafile(SITE_HDF5_FILE, bbox, "elevation")[2]
    else:
        elevation = np.zeros(lons.shape)
    print("---- Identifying site values")
    # Integer number of multiples of spc indicates the x and y locations
    dx = ((site_lons - (lons[0] - spc / 2.)) / spc).astype(int)
    dy = (np.fabs((site_lats - (lats[0] + spc / 2.)) / spc)).astype(int)
    dx_vs = ((site_lons - (lons_vs[0] - spc / 2.)) / spc).astype(int)
    dy_vs = (np.fabs((site_lats - (lats_vs[0] + spc / 2.)) / spc)).astype(int)
    # Get backarc distance
    print("---- Getting volcanic distance")
    xvf = interpolate_xvf_grid(site_lons, site_lats)

    # Return the slope and geology values at the selected locations
    if as_dataframe:
        print("---- Building dataframe")
        elevation = elevation[dy, dx]
        idx = elevation >= -5.0
        return gpd.GeoDataFrame({
            "geometry": gpd.GeoSeries([
                Point(lon, lat)
                for lon, lat in zip(site_lons[idx], site_lats[idx])]),
            "lon": site_lons[idx],
            "lat": site_lats[idx],
            "slope": slope[dy, dx][idx],
            "vs30": vs30[dy_vs, dx_vs][idx],
            "geology": [GEOL_DICT_KEY_TO_NAME[geol]
                        for geol in geology[dy, dx][idx]],
            "xvf": xvf[idx]})
    idx = elevation >= -5.0
    return slope[dy, dx], vs30[dy, dx], geology[dy, dx], xvf, idx

def from_csv(cls, filename, sep=",", **kwargs):
        """
        Builds a site model object from a csv file
        """
        site_model_input = pd.read_csv(filename, sep=",")
        site_model = gpd.GeoDataFrame({
            "geometry": gpd.GeoSeries(
                [Point(lon, lat) for lon, lat in zip(site_model_input["lon"].values,
                                                     site_model_input["lat"].values)]
                ),
            }
        )

        for key in cls.REQUIRED_HEADERS:
            if key not in site_model_input.columns:
                print("Required attribute %s missing from file - may not "
                      "function as needed" % key)
            else:
                site_model[key] = site_model_input[key]
        return cls(site_model, **kwargs)
    
def slice_wgs84_datafile(dbfile, bounds, datakey, flatten=False):
    """
    For data stored in an hdf5 data file, slice out the data set within
    a bounding box

    :param str dbfile:
        Path to the database file
    :param list bounds:
        Bounding box define as [llon, llat, ulon, ulat]
    :param str datakey:
        Name of the dataset to be sliced
    :param bool flatten:
        If True then flattens the sliced data to three 1-D vectors of lon, lat
        and data, otherwise returns the lons and lats as 1-D vectors of
        (nlon,) and (nlat,) respectively and data as a 2-D vector (nlat, nlon)
    """
    llon, llat, ulon, ulat = tuple(bounds)
    db = h5py.File(dbfile, "r")
    lons = db["longitude"][:]
    lats = db["latitude"][:]
    idx_lon = np.logical_and(lons >= llon, lons <= ulon).nonzero()[0]
    idx_lat = np.logical_and(lats >= llat, lats <= ulat).nonzero()[0]
    lons = lons[idx_lon]
    lats = lats[idx_lat]
    data = db[datakey][idx_lat, :][:, idx_lon]
    db.close()
    if flatten:
        lons, lats = np.meshgrid(lons, lats)
        return lons.flatten(), lats.flatten(), data.flatten()
    else:
        return lons, lats, data
    
def interpolate_xvf_grid(target_lons, target_lats, default_xvf=150.):
    """
    Defines the volcanic front distance(xvf) at a set of locations by direct 2D
    linear interpolation from the 5 arc minute grid
    :param np.ndrray target_lons:
        Vector (1D) of longitudes
    :param np.ndarray target_lats:
        Vector (1D) of latitudes
    :param float default_xvf:
        Default volcanic front distance (km)
    """

    # Define the bounding box of the region - with a 10 arc minute buffer
    spc = 1. / 6.
    bbox = (np.min(target_lons) - spc, np.min(target_lats) - spc,
            np.max(target_lons) + spc, np.max(target_lats) + spc)
    # Extract the slice of the 5' volcanic distance grid
    lons, lats, xvf = slice_wgs84_datafile(VF_DIST_FILE, bbox, "xvf")
    if np.all(np.fabs(xvf - default_xvf) < 1.0E-3):
        # No relevant volcanic distance within bounding box
        return default_xvf * np.ones(target_lons.shape)
    # Interpolate from regular grid to target sites
    lons, lats = np.meshgrid(lons, lats)
    ipl = RegularGridInterpolator((lons[0, :], lats[:, 0][::-1]),
                                  xvf[::-1, :].T, bounds_error=False,
                                  fill_value=default_xvf)
    output_xvf = ipl(np.column_stack([target_lons.flatten(),
                                      target_lats.flatten()]))
    return np.reshape(output_xvf, target_lons.shape)

In [8]:
input_file=file_stations

onshore_only = True
print("Loading locations from %s" % input_file)
df = pd.read_csv(input_file, sep=",")
has_lonlat = ("lon" in df.columns) and ("lat" in df.columns)
if not has_lonlat:
    # Check if inputs in xcoord, ycoord
    has_crds = ("xcoord" in df.columns) and ("ycoord" in df.columns)
    if has_crds:
        df["lon"] = df["xcoord"].values
        df["lat"] = df["ycoord"].values
    else:
        raise ValueError("Input file %s has no lon,lat or "
                         "xcoord,ycoord attributes" % args.input_file)
# Dropping duplicate sites
df.drop_duplicates(["lon", "lat"], inplace=True)
print("Building site model for points")
site_manager = SiteManager.site_model_from_points(df["lon"].values, df["lat"].values, onshore_only=onshore_only)

# Save file

site_manager.to_csv(aux_file)
df = pd.read_csv(aux_file)
df = df.assign(row_number=range(len(df)))
df
df['custom_site_id'] = 's_' + df['row_number'].astype(str)
df=df.drop(columns=['row_number'])
df.to_csv(output_file_stations,index=False)

os.remove(aux_file)

Loading locations from ../Spain/DRAFT_20110511_M5.1_Lorca/OpenQuake_gmfs/site_model_stations.csv
Building site model for points
---- Retrieving data within bounding box
---- Identifying site values
---- Getting volcanic distance
---- Building dataframe
Assigning the sites to the ESHM20 regions
Exporting model to csv file ../Spain/DRAFT_20110511_M5.1_Lorca/OpenQuake_gmfs/aux_file.csv


/Users/chago/openquake/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/chago/openquake/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/chago/openquake/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When 

In [9]:
input_file=file_site_model

onshore_only = True
print("Loading locations from %s" % input_file)
df = pd.read_csv(input_file, sep=",")
has_lonlat = ("lon" in df.columns) and ("lat" in df.columns)
if not has_lonlat:
    # Check if inputs in xcoord, ycoord
    has_crds = ("xcoord" in df.columns) and ("ycoord" in df.columns)
    if has_crds:
        df["lon"] = df["xcoord"].values
        df["lat"] = df["ycoord"].values
    else:
        raise ValueError("Input file %s has no lon,lat or "
                         "xcoord,ycoord attributes" % args.input_file)
# Dropping duplicate sites
df.drop_duplicates(["lon", "lat"], inplace=True)
print("Building site model for points")
site_manager = SiteManager.site_model_from_points(df["lon"].values, df["lat"].values, onshore_only=onshore_only)

# Save file

site_manager.to_csv(aux_file)
df = pd.read_csv(aux_file)
df = df.assign(row_number=range(len(df)))
df
df['custom_site_id'] = 't_' + df['row_number'].astype(str)
df=df.drop(columns=['row_number'])
df.to_csv(output_file_site_model,index=False)

os.remove(aux_file)

Loading locations from ../Spain/DRAFT_20110511_M5.1_Lorca/OpenQuake_gmfs/site_model.csv
Building site model for points
---- Retrieving data within bounding box
---- Identifying site values
---- Getting volcanic distance
---- Building dataframe
Assigning the sites to the ESHM20 regions
Exporting model to csv file ../Spain/DRAFT_20110511_M5.1_Lorca/OpenQuake_gmfs/aux_file.csv


/Users/chago/openquake/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/chago/openquake/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/chago/openquake/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When 